# Predicting Titanic survivors

In [42]:
data=sqlContext.read.csv('file:///E:/Kapil/software-study/Big_Data_NoSql/machine-learning/train.csv',header=True)

In [72]:
data.show(4)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
only showing top 4 rows



In [95]:
##Preparing features and adding 
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier
##Try to collect other features
featureColumns=['Pclass','Sex','SibSp','Parch']
##check stratergies instead of skipping null values.This will improve the model
strInds=[StringIndexer(inputCol=c,outputCol='stringFeature_'+c,handleInvalid='skip') for c in featureColumns]
strInds.append(StringIndexer(inputCol='Survived',outputCol='Feature_Survived'))

##Assembling feature vector from feature columns
ve=VectorAssembler()
ve.setInputCols(['stringFeature_'+c for c in featureColumns])
ve.setOutputCol('Features')
##Using decision tree classfier
lr= DecisionTreeClassifier()
lr.setMaxBins(200)
lr.setLabelCol("Feature_Survived").setFeaturesCol("Features")

strInds.append(ve)
strInds.append(lr)

##Ml pipeline
pipeline=Pipeline()
pipeline.setStages(value=strInds)
##Training dataset
pipelinemodel=pipeline.fit(dataset=data)

In [97]:
##Predicting using testing dataset
testData=sqlContext.read.csv('file:///E:/Kapil/software-study/Big_Data_NoSql/machine-learning/test.csv',header=True)
predictions=pipelinemodel.transform(testData)
predicted=predictions.select(['PassengerId','Prediction'])
##Saving predrictions to file
predicted.write.format("com.databricks.spark.csv").option("header", "true").save("file:///E:/Kapil/software-study/Big_Data_NoSql/machine-learning/predicted.csv")